In [1]:
import pandas as pd
import numpy as np
import time
import matplotlib.pyplot as plt
from statistics import mean
import scipy.io as sio

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.signal import butter, filtfilt, spectrogram, iirnotch
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.signal import butter, filtfilt, spectrogram, iirnotch
import glob
import os
import sys
from scipy import signal
import matplotlib.pyplot as plt
from multiprocessing import Pool

import mne
from mpl_toolkits.axes_grid1 import make_axes_locatable, ImageGrid
from mne.io import concatenate_raws
from mne.io import concatenate_raws
from mne import Epochs, pick_types, events_from_annotations
from mne.preprocessing import create_ecg_epochs, create_eog_epochs, ICA
from mne.channels import read_layout

In [2]:
### File_name
#SIT IM
file_sit_1 = 'recording_eeg_2020-02-17-_1_sit.csv'
file_sit_2 = 'recording_eeg_2020-02-17-_3_sit.csv'
file_sit_3 = 'recording_eeg_2020-02-17-_5_sit.csv'
file_sit_4 = 'recording_eeg_2020-02-17-_7_sit.csv'
file_sit_5 = 'recording_eeg_2020-02-17-_9_sit.csv'
#STAND IM
file_stand_1 = 'recording_eeg_2020-02-17-_2_stand.csv'
file_stand_2 = 'recording_eeg_2020-02-17-_4_stand.csv'
file_stand_3 = 'recording_eeg_2020-02-17-_6_stand.csv'
file_stand_4 = 'recording_eeg_2020-02-17-_8_stand.csv'
file_stand_5 = 'recording_eeg_2020-02-17-_10_stand.csv'

In [3]:
### Read data from csv file.
colnames = [x for x in range(69)]
time_read = time.time()
###### SIT #######
sit_raw_1 = pd.read_csv(file_sit_1, names = colnames)
sit_raw_2 = pd.read_csv(file_sit_2, names = colnames)
sit_raw_3 = pd.read_csv(file_sit_3, names = colnames)
sit_raw_4 = pd.read_csv(file_sit_4, names = colnames)
sit_raw_5 = pd.read_csv(file_sit_5, names = colnames)
###### STAND ######
stand_raw_1 = pd.read_csv(file_stand_1, names = colnames)
stand_raw_2 = pd.read_csv(file_stand_2, names = colnames)
stand_raw_3 = pd.read_csv(file_stand_3, names = colnames)
stand_raw_4 = pd.read_csv(file_stand_4, names = colnames)
stand_raw_5 = pd.read_csv(file_stand_5, names = colnames)
###################
print('read time = ', time.time() - time_read)
raw_list = [sit_raw_1, sit_raw_2, sit_raw_3, sit_raw_4, sit_raw_5, stand_raw_1, stand_raw_2, stand_raw_3, stand_raw_4, stand_raw_5]
time_delete = time.time()
for raw in raw_list:
    del raw[63]
print('deleted time = ', time.time() - time_delete)

read time =  56.817259788513184
deleted time =  3.7400710582733154


In [4]:
def gen_trial(trigger):
    count_trial = 0
    keep_trial = np.zeros(len(trigger))
    keep_trig = 3
    count_round = 0
    for trig_ar in np.array(trigger):
        trig = int(trig_ar)
        if trig == 3 and trig != keep_trig:
            count_trial += 1
        keep_trig = trig
        keep_trial[count_round] = count_trial
        count_round += 1
    max_trial = max(keep_trial)
    if max_trial == 41:
        for i in range(len(keep_trial)):
            if keep_trial[i] > 0:
                keep_trial[i] = keep_trial[i] - 1
    return pd.DataFrame(keep_trial)

def insert_trial(raw_data, col_name):
    raw_trig = raw_data.loc[:, [68]]
    raw_trial = gen_trial(raw_trig)
    raw_data.insert(68, col_name, raw_trial)
    print("Length of trial = Length of data >>>> ", len(raw_trial) == len(raw_data))
    
def fil_data(raw_data, trial):
    fix_data = raw_data.loc[(raw_data[68] == 3) & (raw_data[69] == trial), 1:65]
    sit_data = raw_data.loc[(raw_data[68] == 4) & (raw_data[69] == trial), 1:65]
    stand_data = raw_data.loc[(raw_data[68] == 5) & (raw_data[69] == trial), 1:65]
    resting_data = raw_data.loc[(raw_data[68] == 0) & (raw_data[69] == trial), 1:65]
    sit_status = raw_data.loc[(raw_data[68] == 4) & (raw_data[69] == trial), 68]
    ####Check state####
    if len(sit_status) == 0:
        #stand
        state = 1
    else:
        #sit
        state = 0
    return fix_data, sit_data, stand_data, resting_data, state

def all_trial_in_dict(raw_data, type_data):
    state = np.zeros(40)
    all_fix = {}
    all_sit = {}
    all_stand = {}
    all_state = {}
    all_sit_stand = {}
    for i in range(1, 41):
        each_fix, each_sit, each_stand, status = fil_data(raw_data, i)
        all_stand.update({i:each_stand})
        all_sit.update({i:each_sit})
        all_fix.update({i:each_fix})
        state[i-1] = int(status)
        if int(status) == 0:
            all_sit_stand.update({i:each_sit})
        elif int(status) == 1:
            all_sit_stand.update({i:each_stand})
        if i == 40:
            all_state.update({'state':state.reshape(40, 1)})
    if type_data == 'mr':
        return all_sit, all_stand, all_state
    elif type_data == 'mi':
        return all_sit_stand, all_state

def all_trial_in_array(raw_data, type_data):
    
    all_fix = np.zeros((40, 64, 3600))
    all_sit = np.zeros((20, 64, 4800))
    all_stand = np.zeros((20, 64, 4800))
    
    all_sit_stand = np.zeros((40, 64, 4800))
    
    all_fix_1s_and_sit = np.zeros((20, 64, 6000))
    all_fix_1s_and_stand = np.zeros((20, 64, 6000))
    all_fix_1s_and_sit_stand = np.zeros((40, 64, 6000))
    
    all_resting = np.zeros((40, 64, 6000))
    all_sit_resting = np.zeros((20, 64, 6000))
    all_stand_resting = np.zeros((20, 64, 6000))
    
    all_state = np.zeros(40)
    
    sit_index = 0
    stand_index = 0
    
    for i in range(1, 41):
        
        each_fix, each_sit, each_stand, each_resting, status = fil_data(raw_data, i)
        
        if each_resting.shape[0] < 1200 * 5:
            print('Stand data lost some sampling at trial::::', i, 'with size::::', each_resting.shape[0])
            each_resting = mean_padding(each_resting)
                ########Padding data with 100 hundred means points before#######

        each_fix_1s = each_fix[-1200:]
        each_fix = each_fix[0:3600]
        each_sit = each_sit[0:4800]
        each_stand = each_stand[0:4800]
        each_resting = each_resting[0:6000]
        
        ## APPEND IN ARRAY ###
        if int(status) == 0:
            
            all_sit[sit_index] = each_sit.T
            all_sit_stand[i-1] = each_sit.T
            
            all_fix_1s_and_sit_stand[i-1] = np.concatenate((each_fix_1s, each_sit), axis=0).T
            all_fix_1s_and_sit[sit_index] = np.concatenate((each_fix_1s, each_sit), axis=0).T
            
            all_sit_resting[sit_index] = each_resting.T
            sit_index += 1
        else:
            
            all_stand[stand_index] = each_stand.T
            all_sit_stand[i-1] = each_stand.T
            
            all_fix_1s_and_sit_stand[i-1] = np.concatenate((each_fix_1s, each_stand), axis=0).T
            all_fix_1s_and_stand[stand_index] = np.concatenate((each_fix_1s, each_stand), axis=0).T
            
            all_stand_resting[stand_index] = each_resting.T
            stand_index += 1
            
        all_fix[i-1] = each_fix.T
        all_resting[i-1] = each_resting.T
        all_state[i-1] = int(status)
    if type_data == 'mr':
        return all_sit, all_stand, all_state
    elif type_data == 'mi':
        return all_fix_1s_and_sit, all_fix_1s_and_stand, all_sit_resting, all_stand_resting
    
def mean_padding(data_with_padding):
    data_with_padding = np.array(data_with_padding)
    ####Padding with mean 100 points before.####
    padding_size = 6000 - data_with_padding.shape[0]
    keep_point = np.zeros((100, 64))
    value_arr = np.zeros(64)
    for idx, row in enumerate(range(data_with_padding.shape[0], data_with_padding.shape[0] - 100, -1)):
        keep_point[idx, :] = data_with_padding[row-1, :]
        
    for i in range(64):
        value_arr[i] = mean(keep_point[:, i])

    
    for i in range(6000 - data_with_padding.shape[0]):
        data_with_padding = np.insert(data_with_padding, data_with_padding.shape[0], value_arr, axis=0)
    print('Padding with ', value_arr)
    print('Mean padding finished.', value_arr.shape, np.transpose(data_with_padding).shape)
    
    return data_with_padding

def chk_array_data(data_1, data_2):
    pass
    count_round = 0
    for i in range(0, 20):
        if count_round == 0:
            print("-----------------Equality between data_1 and stand data_2 "+ str(i) +"-----------------")
            print(sit_data[i] == stand_data[i])
        else:
            print("-----------------Equality between data_1 and stand data_2 "+ str(i) +"-----------------")
            print(sit_data[i] == stand_data[i])
            print("-----------------Equality between trial "+str(i)+' and '+str(i-1)+"-----------------")
            print("-----DATA_1-----")
            print(sit_data[i] == sit_data[i-1])
            print("-----DATA_2-----")
            print(stand_data[i] == stand_data[i-1])
        count_round += 1

def concatenate_all_trial(data):
    print('Concatenating...')
    for tri_numb, each_tri in enumerate(data):
        if tri_numb == 0:
            passed_tri = each_tri
        else:
            all_trial2D = np.concatenate((passed_tri, each_tri), axis=1)
            passed_tri = all_trial2D
    print('Finished ! ')
    return all_trial2D

def all_run_data(data_run_1, data_run_2, data_run_3, data_run_4, data_run_5):
    all_run_data_arr = np.concatenate((data_run_1, data_run_2, data_run_3, data_run_4, data_run_5), axis=0)
    print('Finished !', all_run_data_arr.shape)
    return all_run_data_arr

def t_d(data, duration):
    len_data = len(data)
    samples = []
    for i in range(0, len_data, duration):
        sample = data.values[i:i + duration]
        samples.append(sample)
    array_data = np.array(samples)
    return array_data

# INSERT TRIAL COLUMN

In [5]:
time_insert = time.time()
for raw in raw_list:
    insert_trial(raw, 69)
print("Insert time = ", time.time() - time_insert)
print("Inserted trigger column sit_raw_ 1 >> 5 and stand_raw_ 1 >> 5.")

Length of trial = Length of data >>>>  True
Length of trial = Length of data >>>>  True
Length of trial = Length of data >>>>  True
Length of trial = Length of data >>>>  True
Length of trial = Length of data >>>>  True
Length of trial = Length of data >>>>  True
Length of trial = Length of data >>>>  True
Length of trial = Length of data >>>>  True
Length of trial = Length of data >>>>  True
Length of trial = Length of data >>>>  True
Insert time =  3.250792980194092
Inserted trigger column sit_raw_ 1 >> 5 and stand_raw_ 1 >> 5.


# SHAPE DATA

In [6]:
#### SIT IMAGERY ####
sit_sit_1, sit_stand_1, sit_sit_rst_1, sit_stand_rst_1 = all_trial_in_array(sit_raw_1, 'mi')
sit_sit_2, sit_stand_2, sit_sit_rst_2, sit_stand_rst_2 = all_trial_in_array(sit_raw_2, 'mi')
sit_sit_3, sit_stand_3, sit_sit_rst_3, sit_stand_rst_3 = all_trial_in_array(sit_raw_3, 'mi')
sit_sit_4, sit_stand_4, sit_sit_rst_4, sit_stand_rst_4 = all_trial_in_array(sit_raw_4, 'mi')
sit_sit_5, sit_stand_5, sit_sit_rst_5, sit_stand_rst_5 = all_trial_in_array(sit_raw_5, 'mi')

##### STAND IMAGERY ####
stand_sit_1, stand_stand_1, stand_sit_rst_1, stand_stand_rst_1 = all_trial_in_array(stand_raw_1, 'mi')
stand_sit_2, stand_stand_2, stand_sit_rst_2, stand_stand_rst_2 = all_trial_in_array(stand_raw_2, 'mi')
stand_sit_3, stand_stand_3, stand_sit_rst_3, stand_stand_rst_3 = all_trial_in_array(stand_raw_3, 'mi')
stand_sit_4, stand_stand_4, stand_sit_rst_4, stand_stand_rst_4 = all_trial_in_array(stand_raw_4, 'mi')
stand_sit_5, stand_stand_5, stand_sit_rst_5, stand_stand_rst_5 = all_trial_in_array(stand_raw_5, 'mi')

Stand data lost some sampling at trial:::: 5 with size:::: 5996
Padding with  [-278117.82906 -280794.63783 -279718.00463 -290255.11095 -284634.29498
 -277328.28097 -282450.88316 -281282.82598 -279943.57592 -290573.59532
 -285733.1466  -283218.58998 -280340.44967 -280261.39589 -279713.55781
 -282165.81316 -283022.30248 -286332.34718 -274911.16001 -287269.22433
 -285587.19473 -285769.13966 -283886.3694  -284342.30684 -280437.32905
 -282909.61532 -292641.62616 -287418.03404 -282831.38971 -282927.21282
 -274957.58248 -286579.3825  -285744.21472 -285697.56657 -279864.29091
 -290338.60374 -283841.011   -284617.50064 -276828.64844 -278209.78815
 -284060.26246 -276997.82721 -278034.38591 -286852.39157 -291569.55566
 -280127.61151 -285156.2691  -280221.90001 -285427.55224 -292378.52535
 -291011.53662 -288695.79314 -287214.57939 -285697.105   -280460.36717
 -279424.35218 -280249.0197  -283613.76994 -284013.99871 -284583.73251
 -282674.06308 -276575.56029 -245527.46484 -381604.71969]
Mean padding

# ALL TRIAL FROM 5 RUNS

In [7]:
########### SIT ###########
sit_sitim = all_run_data(sit_sit_1, sit_sit_2, sit_sit_3, sit_sit_4, sit_sit_5)
sit_sitrst = all_run_data(sit_sit_rst_1, sit_sit_rst_2, sit_sit_rst_3, sit_sit_rst_4, sit_sit_rst_5)
sit_standim = all_run_data(sit_stand_1, sit_stand_2, sit_stand_3, sit_stand_4, sit_stand_5)
sit_standrst = all_run_data(sit_stand_rst_1, sit_stand_rst_2, sit_stand_rst_3, sit_stand_rst_4, sit_stand_rst_5)
########### STAND ##########
stand_sitim = all_run_data(stand_sit_1, stand_sit_2, stand_sit_3, stand_sit_4, stand_sit_5)
stand_sitrst = all_run_data(stand_sit_rst_1, stand_sit_rst_2, stand_sit_rst_3, stand_sit_rst_4, stand_sit_rst_5)
stand_standim = all_run_data(stand_stand_1, stand_stand_2, stand_stand_3, stand_stand_4, stand_stand_5)
stand_standrst = all_run_data(stand_stand_rst_1, stand_stand_rst_2, stand_stand_rst_3, stand_stand_rst_4, stand_stand_rst_5)

Finished ! (100, 64, 6000)
Finished ! (100, 64, 6000)
Finished ! (100, 64, 6000)
Finished ! (100, 64, 6000)
Finished ! (100, 64, 6000)
Finished ! (100, 64, 6000)
Finished ! (100, 64, 6000)
Finished ! (100, 64, 6000)


# FILTER AND ICA

In [8]:
ch_names = ["AFz", "Fz", "Fp1", "Fp2", "AF7", "AF5", "AF3", "AF4", "AF6", "AF8", "F7", 
            "F5", "F3", "F1", "F2", "F4", "F6", "F8", "FT7", "FC5", "FC3", "FC1", "FCz", 
            "FC2", "FC4", "FC6", "FT8", "T7", "C5", "C3", "C1", "Cz", "C2", "C4", "C6", 
            "T8","TP7", "CP5", "CP3", "CP1", "CPz", "CP2", "CP4", "CP6", "TP8", "P7", 
            "P5", "P3", "P1", "Pz", "P2", "P4", "P6", "P8", "PO7", "PO3", "POz", "PO4", 
            "PO8", "O1", "Oz", "O2"]
ch_eogs = ["VEOG", "HEOG"]
ch_names_all = ch_names+ch_eogs
smp_freq = 1200
print(len(ch_names))

62


In [9]:
# Preprocessing
def butter_bandpass_filter(data, lowcut, highcut, fs, order):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    return y
def notch_filter(data, f0, fs, Q): # Q = Quality factor
    w0 = f0/(fs/2)
    b, a = iirnotch(w0, Q)
    y = filtfilt(b, a, data)
    return y
def preprocessing(data):
    """Preprocessing time-dimain EEG raw data

    Parameters
    ----------
    data : EEG time-domain raw data 

    Returns
    ----------
    filtered : filtered EEG time-domain raw data 
    """
    filtered = notch_filter(data, 50, smp_freq, 25)
    filtered = butter_bandpass_filter(filtered, 0.05, 40, smp_freq, 2)
    print('notch_filter 50 Hz .... bandpass 0.05-40 Hz.')
    return filtered

In [10]:
def calculate_ICA(arr, save_name='SUB_01', threshold=2):
    """
    Parameters
    ----------
    arr : output from collect_data_allphase
    save_name : figure name, keep none if no need to save
    return:
        arr liked array with EOG removal
    """
    if save_name is not None:
        for directory in ['ica','eog_score','eog_avg','raw_EEG','corrected_EEG','montage','new_raw']:
            if not os.path.exists(directory):
                os.makedirs(directory)

    # Create a dummy mne.io.RawArray object
#     ch_types = ['eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eeg', 'eog', 'eog']
    ch_names = ["AFz", "Fz", "Fp1", "Fp2", "AF7", "AF5", "AF3", "AF4", "AF6", "AF8", "F7", 
            "F5", "F3", "F1", "F2", "F4", "F6", "F8", "FT7", "FC5", "FC3", "FC1", "FCz", 
            "FC2", "FC4", "FC6", "FT8", "T7", "C5", "C3", "C1", "Cz", "C2", "C4", "C6", 
            "T8","TP7", "CP5", "CP3", "CP1", "CPz", "CP2", "CP4", "CP6", "TP8", "P7", 
            "P5", "P3", "P1", "Pz", "P2", "P4", "P6", "P8", "PO7", "PO3", "POz", "PO4", 
            "PO8", "O1", "Oz", "O2"]
    ch_eogs = ["VEOG", "HEOG"]
    ch_names_all = ch_names+ch_eogs
    ch_types = ['eeg' for i in range(len(ch_names))] + ['eog', 'eog']
    # Create an info object.
    info = mne.create_info(ch_names=ch_names_all, sfreq=smp_freq, ch_types=ch_types)
    raw = mne.io.RawArray(arr, info)
    raw.set_montage("standard_1020")
    
    ica = ICA(method = "fastica", random_state=42)
    ica.fit(raw)

    n_max_eog =  5  # use max 3 components
    eog_epochs = create_eog_epochs(raw)
    eog_epochs.apply_baseline((None, None))

    try:
        print('debug !')
        eog_inds, scores_eog = ica.find_bads_eog(eog_epochs,threshold=threshold)
        print('Found %d EOG component(s)' % (len(eog_inds),))

        #remove EOG from EEG
        ica.exclude += eog_inds
    except:
        pass
    print('debug !')
    raw_corrected = raw
    ica.apply(raw_corrected)
    print(ica)

    # #plot
#     if save_name is not None:
#         print('debug !')
#         ica.plot_sources(raw, show = False).savefig('ica/'+save_name+'_ica.png') 
#         try:
#             ica.plot_scores(scores_eog, exclude=eog_inds, title='EOG scores',show = False).savefig('eog_score/'+save_name+'_eog_score.png') 
#         except:
#             pass
#         scalings = 'auto'
#         ica.plot_sources(eog_epochs.average(), title='EOG average',show = False).savefig('eog_avg/'+save_name+'_eog_avg.png')
#         raw.plot(show = False,scalings=scalings).savefig('raw_EEG/'+save_name+'_raw_EEG.png')
#         raw_corrected.plot(show = False,scalings=scalings).savefig('corrected_EEG/'+save_name+'_corrected_EEG.png')
#         ica.plot_components(inst=raw, show = False)[0].savefig('montage/'+save_name+'_montage.png')
#         print('======================================')
#         print(raw_corrected.get_data().shape)
#         raw_corrected.save('new_raw/'+save_name+'_raw.fif', overwrite=True)
    return raw_corrected

# PREPROCESS ALL TRIAL

In [11]:
######## SIT #########
sit_sit_pre = np.zeros(sit_sitim.shape)
sit_sitrst_pre = np.zeros(sit_sitrst.shape)
sit_stand_pre = np.zeros(sit_standim.shape)
sit_standrst_pre = np.zeros(sit_standrst.shape)

######## STAND ########
stand_sit_pre = np.zeros(stand_sitim.shape)
stand_sitrst_pre = np.zeros(stand_sitrst.shape)
stand_stand_pre = np.zeros(stand_standim.shape)
stand_standrst_pre = np.zeros(stand_standrst.shape)

for trial in range(sit_sit_pre.shape[0]):
    sit_sit_pre[trial] = preprocessing(sit_sitim[trial])
    sit_sitrst_pre[trial] = preprocessing(sit_sitrst[trial])
    sit_stand_pre[trial] = preprocessing(sit_standim[trial])
    sit_standrst_pre[trial] = preprocessing(sit_standrst[trial])
    stand_sit_pre[trial] = preprocessing(stand_sitim[trial])
    stand_sitrst_pre[trial] = preprocessing(stand_sitrst[trial])
    stand_stand_pre[trial] = preprocessing(stand_standim[trial])
    stand_standrst_pre[trial] = preprocessing(stand_standrst[trial])

notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filt

notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filt

notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filt

notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filt

notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filter 50 Hz .... bandpass 0.05-40 Hz.
notch_filt

# CALCULATE ICA ALL TRIAL

In [12]:
sit_sit_ica = np.zeros(sit_sit_pre.shape)
sit_sitrst_ica = np.zeros(sit_sitrst_pre.shape)
sit_stand_ica = np.zeros(sit_stand_pre.shape)
sit_standrst_ica = np.zeros(sit_standrst.shape)

stand_sit_ica = np.zeros(stand_sit_pre.shape)
stand_sitrst_ica = np.zeros(stand_sitrst_pre.shape)
stand_stand_ica = np.zeros(stand_stand_pre.shape)
stand_standrst_ica = np.zeros(stand_standrst_pre.shape)

for eachtri in range(100):
    sit_sit_ica[eachtri] = calculate_ICA(sit_sit_pre[eachtri])[:][0]
    sit_sitrst_ica[eachtri] = calculate_ICA(sit_sitrst_pre[eachtri])[:][0]
    sit_stand_ica[eachtri] = calculate_ICA(sit_stand_pre[eachtri])[:][0]
    sit_standrst_ica[eachtri] = calculate_ICA(sit_standrst_pre[eachtri])[:][0]
    
    stand_sit_ica[eachtri] = calculate_ICA(stand_sit_pre[eachtri])[:][0]
    stand_sitrst_ica[eachtri] = calculate_ICA(stand_sitrst_pre[eachtri])[:][0]
    stand_stand_ica[eachtri] = calculate_ICA(stand_stand_pre[eachtri])[:][0]
    stand_standrst_ica[eachtri] = calculate_ICA(stand_standrst_pre[eachtri])[:][0]

Creating RawArray with float64 data, n_channels=64, n_times=6000
    Range : 0 ... 5999 =      0.000 ...     4.999 secs
Ready.
DigMontage is a superset of info. 32 in DigMontage will be ignored. The ignored channels are: {'TP10', 'AF2', 'Fpz', 'T4', 'F9', 'AF10', 'Iz', 'PO9', 'PO10', 'PO6', 'O10', 'TP9', 'PO2', 'M1', 'P9', 'T5', 'M2', 'FT10', 'T9', 'PO5', 'AF9', 'P10', 'A1', 'A2', 'T10', 'T3', 'F10', 'AF1', 'O9', 'FT9', 'PO1', 'T6'}
Fitting ICA to data using 62 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 62
Fitting ICA took 2.8s.
EOG channel index for this subject is: [62 63]
Filtering the data to remove DC offset to help distinguish blinks from saccades


<ipython-input-10-a4a55df8a7dc>:35: RuntimeWarning: filter_length (16384) is longer than the signal (6000), distortion is likely. Reduce filter length or filter a longer signal.
  eog_epochs = create_eog_epochs(raw)
<ipython-input-10-a4a55df8a7dc>:35: RuntimeWarning: filter_length (16384) is longer than the signal (6000), distortion is likely. Reduce filter length or filter a longer signal.
  eog_epochs = create_eog_epochs(raw)


Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 16384 samples (13.653 sec)



<ipython-input-10-a4a55df8a7dc>:35: RuntimeWarning: filter_length (16384) is longer than the signal (6000), distortion is likely. Reduce filter length or filter a longer signal.
  eog_epochs = create_eog_epochs(raw)


Now detecting blinks and generating corresponding events
Found 10 significant peaks
Number of EOG events detected : 10
10 matching events found
No baseline correction applied
Not setting metadata
Loading data for 10 events and 1201 original time points ...
2 bad epochs dropped
Applying baseline correction (mode: mean)
debug !
Found 9 EOG component(s)
debug !
Transforming to ICA space (62 components)
Zeroing out 9 ICA components
<ICA  |  raw data decomposition, fit (fastica): 6000 samples, 62 components, channels used: "eeg", 9 sources marked for exclusion>
Creating RawArray with float64 data, n_channels=64, n_times=6000
    Range : 0 ... 5999 =      0.000 ...     4.999 secs
Ready.
DigMontage is a superset of info. 32 in DigMontage will be ignored. The ignored channels are: {'TP10', 'AF2', 'Fpz', 'T4', 'F9', 'AF10', 'Iz', 'PO9', 'PO10', 'PO6', 'O10', 'TP9', 'PO2', 'M1', 'P9', 'T5', 'M2', 'FT10', 'T9', 'PO5', 'AF9', 'P10', 'A1', 'A2', 'T10', 'T3', 'F10', 'AF1', 'O9', 'FT9', 'PO1', 'T6'}


...

Now detecting blinks and generating corresponding events
Found 4 significant peaks
Number of EOG events detected : 4
4 matching events found
No baseline correction applied
Not setting metadata
Loading data for 4 events and 1201 original time points ...
2 bad epochs dropped
Applying baseline correction (mode: mean)
debug !
Found 9 EOG component(s)
debug !
Transforming to ICA space (62 components)
Zeroing out 9 ICA components
<ICA  |  raw data decomposition, fit (fastica): 6000 samples, 62 components, channels used: "eeg", 9 sources marked for exclusion>
Creating RawArray with float64 data, n_channels=64, n_times=6000
    Range : 0 ... 5999 =      0.000 ...     4.999 secs
Ready.
DigMontage is a superset of info. 32 in DigMontage will be ignored. The ignored channels are: {'TP10', 'AF2', 'Fpz', 'T4', 'F9', 'AF10', 'Iz', 'PO9', 'PO10', 'PO6', 'O10', 'TP9', 'PO2', 'M1', 'P9', 'T5', 'M2', 'FT10', 'T9', 'PO5', 'AF9', 'P10', 'A1', 'A2', 'T10', 'T3', 'F10', 'AF1', 'O9', 'FT9', 'PO1', 'T6'}
Fitt

<ipython-input-10-a4a55df8a7dc>:35: RuntimeWarning: filter_length (16384) is longer than the signal (6000), distortion is likely. Reduce filter length or filter a longer signal.
  eog_epochs = create_eog_epochs(raw)
<ipython-input-10-a4a55df8a7dc>:35: RuntimeWarning: filter_length (16384) is longer than the signal (6000), distortion is likely. Reduce filter length or filter a longer signal.
  eog_epochs = create_eog_epochs(raw)


Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 16384 samples (13.653 sec)



<ipython-input-10-a4a55df8a7dc>:35: RuntimeWarning: filter_length (16384) is longer than the signal (6000), distortion is likely. Reduce filter length or filter a longer signal.
  eog_epochs = create_eog_epochs(raw)


Now detecting blinks and generating corresponding events
Found 5 significant peaks
Number of EOG events detected : 5
5 matching events found
No baseline correction applied
Not setting metadata
Loading data for 5 events and 1201 original time points ...
2 bad epochs dropped
Applying baseline correction (mode: mean)
debug !
Found 8 EOG component(s)
debug !
Transforming to ICA space (62 components)
Zeroing out 8 ICA components
<ICA  |  raw data decomposition, fit (fastica): 6000 samples, 62 components, channels used: "eeg", 8 sources marked for exclusion>


# Concatenate ICA

In [13]:
sit_sitlab = concatenate_all_trial(sit_sit_ica)
sit_sitrstlab = concatenate_all_trial(sit_sitrst_ica)
sit_standlab = concatenate_all_trial(sit_stand_ica)
sit_standrstlab = concatenate_all_trial(sit_standrst_ica)

stand_sitlab = concatenate_all_trial(stand_sit_ica) 
stand_sitrstlab = concatenate_all_trial(stand_sitrst_ica)
stand_standlab = concatenate_all_trial(stand_stand_ica)
stand_standrstlab = concatenate_all_trial(stand_standrst_ica)

Concatenating...
Finished ! 
Concatenating...
Finished ! 
Concatenating...
Finished ! 
Concatenating...
Finished ! 
Concatenating...
Finished ! 
Concatenating...
Finished ! 
Concatenating...
Finished ! 
Concatenating...
Finished ! 


# SAVE TO ARRAY FILE 

In [21]:
np.save('sitim_during_sitting.npy', sit_sit_ica)
np.save('sitrst_during_sitting.npy', sit_sitrst_ica)
np.save('standim_during_sitting.npy', sit_stand_ica)
np.save('standrst_during_sitting.npy', sit_standrst_ica)

np.save('sitim_during_standing.npy', stand_sit_ica)
np.save('sitrst_during_standing.npy', stand_sitrst_ica)
np.save('standim_during_standing.npy', stand_stand_ica)
np.save('standrst_during_standing.npy', stand_standrst_ica)

In [14]:
print(sit_sitlab.shape)
print(sit_sitrstlab.shape)
print(sit_standlab.shape)
print(sit_standrstlab.shape)

print(stand_sitlab.shape)
print(stand_sitrstlab.shape)
print(stand_standlab.shape)
print(stand_standrstlab.shape)

(64, 600000)
(64, 600000)
(64, 600000)
(64, 600000)
(64, 600000)
(64, 600000)
(64, 600000)
(64, 600000)


# TEST SIT DATA TO MATLAB FILE

In [20]:
############## SIT ################
sio.savemat('S01_sitim_during_sitting.mat', {'data':sit_sitlab})
sio.savemat('S01_sitrst_during_sitting.mat', {'data':sit_sitrstlab})
sio.savemat('S01_standim_during_sitting.mat', {'data':sit_standlab})
sio.savemat('S01_standrst_durring_sitting.mat', {'data':sit_standrstlab})

############## STAND ################
sio.savemat('S01_sitim_during_standing.mat', {'data':stand_sitlab})
sio.savemat('S01_sitrst_during_standing.mat', {'data':stand_sitrstlab})
sio.savemat('S01_standim_during_standing.mat', {'data':stand_standlab})
sio.savemat('S01_standrst_during_standing.mat', {'data':stand_standrstlab})